# GEDI <img align="right" src="../../resources/easi_logo.jpg">

### Index
- [Overview](#Overview)
- [Setup (imports, dask, query parameters)](#Setup)
- [Example query (2D product)](#Example-query-for-the-2D-product)
- [Product definition (measurements, flags)](#Product-definition)
- [Create a mask](#Create-a-mask)
- [Visualisation](#Visualisation)
- [Loading DEM layer from GEDI](#Loading-DEM-layer-from-GEDI)
- [Load GEDI 3D datasets](#Load-GEDI-3D-datasets)

## Overview

The Global Ecosystem Dynamics Investigation (GEDI) produces high resolution laser ranging observations of the 3D structure of the Earth. GEDI's precise measurements of forest canopy height, canopy vertical structure, and surface elevation greatly advance our ability to characterize important carbon and water cycling processes, biodiversity, and habitat.

The GEDI instrument is a geodetic-class, light detection and ranging (lidar) laser system comprised of 3 lasers that produce 8 parallel tracks of observations. Each laser fires 242 times per second and illuminates a 25 m spot (a footprint) on the surface over which 3D structure is measured. Each footprint is separated by 60 m along track, with an across-track distance of about 600 m between each of the 8 tracks. GEDI expected to produce about 10 billion cloud-free observations during its nominal 24-month mission length.

### Data source and documentation

- [Official GEDI web site](https://gedi.umd.edu)
- [Information about the GEDI products](https://gedi.umd.edu/data/products/)

Some Level 2B data has been indexed in EASI, as demonstrated in this notebook.

### EASI pipeline

| Task | Summary 
|------|---------
| Source | https://lpdaacsvc.cr.usgs.gov/services/gedifinder
| Download | https://e4ftl01.cr.usgs.gov/GEDI
| Preprocess | 
| Format | Convert to COGs 
| Prepare | Y 
| TODO | 

## Setup

### Imports

In [ ]:
# Data tools
import numpy as np
import pandas as pd

# Datacube
import datacube

# Visualisation tools
# Ignore shapely deprecation warnings. To be removed at next image update
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

from holoviews import opts
from holoviews.operation.datashader import regrid
import geoviews as gv
from geoviews import tile_sources as gvts

# EASI tools
import sys
sys.path.append('../../scripts')
import notebook_utils

### Dask

In [ ]:
cluster, client = notebook_utils.initialize_dask(workers=(1,2), use_gateway=True)
display(cluster)
display(client)

### ODC database

In [ ]:
# Template code for development database
# CONF = """
# [datacube]
# db_hostname:
# db_database:
# db_username:
# db_password:
# """
# from datacube.config import read_config, LocalConfig
# dc = datacube.Datacube(config=LocalConfig(read_config(CONF)), env='datacube')

dc = datacube.Datacube()  # Comment if using a development database

### Show GEDI products indexed in ODC
 There should be one 2D product (`gedi_l2b`) and three 3D products (`cover_z`, `pai_z`, `pavd_z`).

In [ ]:
# Available products
dc.list_products().filter(like="gedi", axis=0)

### Area of interest and common query parameters
We define an area of interest around Litchfield National Park, and set the projection (CRS) and spatial resolution we will use to fetch data in this notebook.

Change any of these parameters, using the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions  + /product

In [ ]:
query = {
    'latitude': (-14.006031459249373, -12.980471333463804),  # "x" axis bounds
    'longitude': (130.1715087890625, 131.22894287109375),    # "y" axis bounds
    'output_crs': 'EPSG:4236',                               # EPSG code
    'resolution': (-1/3600, 1/3600),                         # Target resolution
}

## Example query for the 2D product
Load the `beam` data from the 2D GEDI product `gedi_l2b`.

In [ ]:
product = 'gedi_l2b'
measurements = ['beam']

data = dc.load(
    product=product,            # Product name
    measurements=measurements,  # Measurements to load
    **query
)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
data

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[product]
notebook_utils.heading(f'Measurement table for product: {product}')
notebook_utils.display_table(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
notebook_utils.display_table(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    notebook_utils.display_table(masking.describe_variable_flags(data[flag]))

## Create a mask

Pixels to keep have `beam` data different from `no-data` for at least one time slice. So we start by fusing the beam data into a single layer, keeping any non-`no-data`.

In [ ]:
def func_combine_dates(da, dask=False):
    """Fuse different dates of GEDI data into single layer."""
    dx = None
    for itime in da.time:
        if (dask):
            dt = da.sel({'time':itime}).compute()
        else:
            dt = da.sel({'time':itime})
        if dx is None:
            dx = dt
        else:
            idx = np.where(dt.values != da.attrs['nodata'])
            dx.values[idx] = dt.values[idx]
    return dx

In [ ]:
beam = func_combine_dates(data['beam'])
beam

Pull the beam values, replacing `no-data` pixels by `NaN` for visualisation purposes.

In [ ]:
mask = beam.values == beam.attrs["nodata"]
beam_vals = beam.values.astype(np.float32)  # Convert to float to support NaN values
beam_vals[mask] = np.nan

## Visualisation
Plot the GEDI  beam data over Open Street Map. The map is zoomable.

In [ ]:
opts.defaults(
    opts.Curve(xaxis=None, yaxis=None, show_grid=False, show_frame=False,
               color='orangered', framewise=True, width=100),
    opts.Image(width=800, height=400, shared_axes=False, colorbar=True,
               xaxis=True, yaxis=True, axiswise=True, bgcolor='white'),
    opts.HLine(color='white', line_width=1),
    opts.Layout(shared_axes=False),
    opts.VLine(color='white', line_width=1))

In [ ]:
gvts.OSM * regrid(
    gv.Image((beam.longitude, beam.latitude, beam_vals))
).opts(
    cmap='jet',
    clim=(0,13),
    width=800,
    height=800
)

## Loading DEM layer from GEDI
The DEM can be loaded and displayed in a similar way.

In [ ]:
product = 'gedi_l2b'
measurements=['digital_elevation_model']

data2 = dc.load(
    product=product,            # Product name
    measurements=measurements,  # Measurements to load
    **query
)
dem = func_combine_dates(data2['digital_elevation_model'])

In [ ]:
dem_vals = dem.values
dem_vals[mask] = np.nan
gvts.OSM * regrid(
        gv.Image((dem.longitude, dem.latitude, dem_vals))
    ).opts(
        title='DEM - Litchfield', 
        clim=(0,400), 
        cmap='jet', 
        colorbar=True, 
        width=800, 
        height=800
    ).hist()

### Unload some data before continuing

In [ ]:
del beam, dem, dem_vals, data, data2

## Load GEDI 3D datasets
There are three 3D datasets: `cover_z`, `pai_z` and `pavd_z` available as separate products. Here we load `cover_z` as an example.

In [ ]:
product='gedi_l2b_cover_z'
# Measurements are the different altitudes, we load them all.
data = dc.load(
    product=product,            # Product name
    dask_chunks={'time': 1},    # Use dask chunks
    **query
)
data

In [ ]:
# Display the second time slice (t=1)
print("Expand Data variables to see the data at each altitude")
data.isel(time=1).load()